In [1]:
import numpy as np
import pandas as pd 
import sklearn.linear_model as skllm
import scipy.stats 

In [2]:
df_train = pd.read_csv("data/ozone_train.csv")
X_train = df_train.loc[:, df_train.columns != "FFVC"]
y_train = df_train.loc[:, "FFVC"]


In [3]:
df_test = pd.read_csv("data/ozone_test.csv")
X_test = df_test.loc[:, df_test.columns != "FFVC"]
y_test = df_test.loc[:, "FFVC"]


In [13]:
# Problem 1.2

def summary_OLS(X, y, feature_names):
    array_1D = None
    
    if len(np.shape(X)) == 1:
        array_1D = True
        X = X.reshape(-1,1)
        
    OLS = skllm.LinearRegression(fit_intercept=True).fit(X, y)
    y_pred = OLS.predict(X) 
    coefficients = np.append(OLS.intercept_, OLS.coef_)
    
    
    newX = np.append(np.ones((len(X),1)), X, axis=1)
    MSE = (sum((y - y_pred)**2))/(len(newX)-len(newX[0]))

    var = MSE*(np.linalg.pinv(np.dot(newX.T,newX)).diagonal())
    std = np.sqrt(var)
    ts_b = coefficients/ std

    p = np.zeros(len(ts_b))
    for index, i in enumerate(ts_b):
        p[index] = 2*(1-scipy.stats.t.cdf(np.abs(i),(len(newX)-1))) 
        
    if array_1D:
        summary = pd.DataFrame(zip(coefficients, std, p), columns=["Feature", "Standard deviance", "p-values"]
                              , index=(["intercept"] + [feature_names]))
        
    else:
        summary = pd.DataFrame(zip(coefficients, std, p), columns=["Coefficients", "Standard deviance", "p-values"],
                          index=(["intercept"] + list(feature_names)))
                           
    return summary
        


In [14]:
summary_train = summary_OLS(X_train, y_train, X_train.columns.values)

summary_test = summary_OLS(X_test, y_test, X_test.columns.values)

summary_train

,Coefficients,Standard deviance,p-values
intercept,0.164510,0.067556,1.559437e-02
ADHEU,-0.075544,0.207669,7.163388e-01
HOCHOZON,-0.177523,0.114152,1.211935e-01
AMATOP,-0.035130,0.102232,7.314208e-01
AVATOP,-0.072781,0.108710,5.038038e-01
ADEKZ,0.023488,0.102991,8.197875e-01
ARAUCH,-0.051278,0.095157,5.904580e-01
FSNIGHT,0.006259,0.159481,9.687241e-01
FMILB,-0.180463,0.167958,2.836679e-01
FTIER,-0.161323,0.166531,3.336307e-01


In [42]:
# Problem 1.3

def backward_elimination(X, y, alpha):
    # Make copies of X and y
    X_copy = X.copy()
    y_copy = y.copy()
    
    feature_names = X_copy.columns.values
    # Calculating coefficients, standard deviations and p-values as summary
    # of initial X and y 
    summary = summary_OLS(X_copy, y_copy, feature_names)
    p_values = summary.loc[:, "p-values"]
    
    # Find maximum p-value
    p_max = np.max(p_values)
    
    # Iterate until p <= alpha
    while p_max > alpha:
        # Calculating coefficients, standard deviations and p-values using summary_OLS function
        feature_names = X_copy.columns.values
        summary = summary_OLS(X_copy, y_copy, feature_names)
        
        p_values = summary.loc[:, "p-values"]
        
        # Locating index with maximum p-value and 
        # dropping corresponding feature
        p_max_index = p_values.idxmax(axis=1)
        X_copy.drop(p_max_index, axis=1, inplace=True)
        
        # Update maximum p-value
        p_max = p_values.loc[p_max_index]        
    feature_names = X_copy.columns.values
    
    return X_copy, summary_OLS(X_copy, y_copy, feature_names)



In [30]:
X_new, summary_new = backward_elimination(X_train, y_train, 1e-1)
summary_new

,Coefficients,Standard deviance,p-values
intercept,3.428418e+13,0.028441,0.000000
FTIER,-3.172860e-01,0.140222,0.024520
FSPFEI,5.158385e-01,0.197393,0.009519
MALE,-3.428418e+13,0.043337,0.000000
FEMALE,-3.428418e+13,0.042557,0.000000
FLGROSS,5.665056e-01,0.056413,0.000000
FLGEW,1.927831e-01,0.056023,0.000680


In [191]:
def forward_selection(X, y, alpha):
    X_copy = X.copy()
    y_copy = y.copy()
    p_max = alpha - 1
    p_min = 0
    feature_include = []
    feature_names = np.array(list(X_copy.columns.values))
    while p_max < alpha and len(feature_names)>0:

        n_features = len(feature_names)
        p_values = np.zeros(n_features)
        
        for i, feature in enumerate(feature_names):
                    
            features = np.append(feature_include, feature_names[i])

            X_feature = X_copy.loc[:, features].values
            summary_ = summary_OLS(X_feature, y_copy, features)

            summary = summary_.drop((["intercept"] + feature_include ), axis=0)
            
            
            p_one = summary.loc[:, "p-values"].values
            p_values[i] = p_one
        
        p_min_index = np.argmin(p_values)
        p_min = p_values[p_min_index]

        feature_min_p = feature_names[p_min_index]
        feature_include.append(feature_min_p)
        feature_names = feature_names[feature_names != feature_min_p]

        
        X_feature = X_copy.loc[:, features].values

        summary_ = summary_OLS(X_feature, y_copy, feature_include)
        
        p_max = summary.loc[:, "p-values"].values.max()



    return feature_include
jeff = forward_selection(X_train, y_train, 1e-2)
jeff

['FLGROSS', 'MALE', 'FEMALE', 'FLGEW', 'FSPFEI']